In [375]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import random
import os

In [376]:
pd.__version__

'1.4.4'

Se a gente lê o arquivo de 3GB inteiro, vai dar problema. Aqui tem uma receita para ler somente x%: https://www.kaggle.com/questions-and-answers/53925 

## Selecionar apenas uma amostra aleatória de 1% dos alunos

In [377]:
def skip(i,fraction=0.01):
    if i == 0:
        return False  # precisamos manter o row=0 para ter o header
    else:
        return random.random() > fraction

[skip(i) for i in range(10)]

[False, True, True, True, True, True, True, True, True, True]

In [378]:
%time df = pd.read_csv('../inep/enem/2010/DADOS/MICRODADOS_ENEM_2010.csv',skiprows = skip, encoding='latin1',sep=";")

CPU times: user 21.4 s, sys: 712 ms, total: 22.1 s
Wall time: 41 s


In [379]:
for col in df.columns:
    print(col)

NU_INSCRICAO
NU_ANO
TP_FAIXA_ETARIA
TP_SEXO
TP_ESTADO_CIVIL
TP_COR_RACA
TP_ST_CONCLUSAO
TP_ENSINO
CO_MUNICIPIO_ESC
NO_MUNICIPIO_ESC
CO_UF_ESC
SG_UF_ESC
TP_DEPENDENCIA_ADM_ESC
TP_LOCALIZACAO_ESC
TP_SIT_FUNC_ESC
IN_CERTIFICADO
CO_MUNICIPIO_PROVA
NO_MUNICIPIO_PROVA
CO_UF_PROVA
SG_UF_PROVA
TP_PRESENCA_CN
TP_PRESENCA_CH
TP_PRESENCA_LC
TP_PRESENCA_MT
CO_PROVA_CN
CO_PROVA_CH
CO_PROVA_LC
CO_PROVA_MT
NU_NOTA_CN
NU_NOTA_CH
NU_NOTA_LC
NU_NOTA_MT
TX_RESPOSTAS_CN
TX_RESPOSTAS_CH
TX_RESPOSTAS_LC
TX_RESPOSTAS_MT
TP_LINGUA
TX_GABARITO_CN
TX_GABARITO_CH
TX_GABARITO_LC
TX_GABARITO_MT
TP_STATUS_REDACAO
NU_NOTA_COMP1
NU_NOTA_COMP2
NU_NOTA_COMP3
NU_NOTA_COMP4
NU_NOTA_COMP5
NU_NOTA_REDACAO
Q01
Q02
Q03
Q04
Q05
Q06
Q07
Q08
Q09
Q10
Q11
Q12
Q13
Q14
Q15
Q16
Q17
Q18
Q19
Q20
Q21
Q22
Q23
Q24
Q25
Q26
Q27
Q28
Q29
Q30
Q31
Q32
Q33
Q34
Q35
Q36
Q37
Q38
Q39
Q40
Q41
Q42
Q43
Q44
Q45
Q46
Q47
Q48
Q49
Q50
Q51
Q52
Q53
Q54
Q55
Q56
Q57


In [380]:
df

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57
0,200000000016,2010,11,M,0,1,1,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200000000233,2010,9,M,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200000000284,2010,7,F,0,0,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200000000398,2010,4,M,0,1,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200000000642,2010,11,F,0,1,1,2.0,NaN,NaN,...,4.0,0.0,0.0,0.0,0.0,D,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46162,200004611468,2010,4,M,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46163,200004611569,2010,14,F,1,3,1,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46164,200004611601,2010,10,M,0,0,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46165,200004611602,2010,4,M,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [381]:
#df.describe()

In [382]:
#df.sample(5)

In [383]:
df["TP_PRESENCA_CN"].value_counts()

1    33877
0    12231
2       59
Name: TP_PRESENCA_CN, dtype: int64

## Limpeza dos dados, selecionando apenas alunos que não faltaram na prova

In [384]:

def filterdf(df):
    if "IN_TREINEIRO" in df.columns:
        df = df[df["TP_PRESENCA_CN"] == 1]  # presente nas 4 provas
        df = df[df["TP_PRESENCA_CH"] == 1]
        df = df[df["TP_PRESENCA_LC"] == 1]
        df = df[df["TP_PRESENCA_MT"] == 1]
        df = df[(df["IN_TREINEIRO"] == 0) | (df["IN_TREINEIRO"].isna())]  # não é treineiro ou não existe 
        df = df[df["TP_ST_CONCLUSAO"].isin([1, 2])]  # afirma que concluiu ou vai concluir EM em 2019
        # não queremos as notas == 0
        df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0") 
    return df

dff = filterdf(df)
dff

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57
0,200000000016,2010,11,M,0,1,1,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200000000233,2010,9,M,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200000000284,2010,7,F,0,0,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200000000398,2010,4,M,0,1,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200000000642,2010,11,F,0,1,1,2.0,NaN,NaN,...,4.0,0.0,0.0,0.0,0.0,D,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46162,200004611468,2010,4,M,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46163,200004611569,2010,14,F,1,3,1,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46164,200004611601,2010,10,M,0,0,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46165,200004611602,2010,4,M,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [385]:
import inspect
filter_conditions = inspect.getsource(filterdf)
print(filter_conditions)

def filterdf(df):
    if "IN_TREINEIRO" in df.columns:
        df = df[df["TP_PRESENCA_CN"] == 1]  # presente nas 4 provas
        df = df[df["TP_PRESENCA_CH"] == 1]
        df = df[df["TP_PRESENCA_LC"] == 1]
        df = df[df["TP_PRESENCA_MT"] == 1]
        df = df[(df["IN_TREINEIRO"] == 0) | (df["IN_TREINEIRO"].isna())]  # não é treineiro ou não existe 
        df = df[df["TP_ST_CONCLUSAO"].isin([1, 2])]  # afirma que concluiu ou vai concluir EM em 2019
        # não queremos as notas == 0
        df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0") 
    return df



## Salvar a amostra filtrada em disco

In [386]:

fn = "data/enem19-1perc"
dff.to_csv(f'{fn}.csv',index=False)
with open(fn+'-filter_conditions.txt','w') as f:
    f.write(filter_conditions)

## Selecionar apenas as colunas que desejamos trabalhar

In [387]:

def selecao(dff):
    df_selecao = dff[["NU_INSCRICAO","NU_ANO","CO_MUNICIPIO_RESIDENCIA","NO_MUNICIPIO_RESIDENCIA","CO_UF_RESIDENCIA","SG_UF_RESIDENCIA","TP_COR_RACA","TP_SEXO","NU_IDADE","TP_ENSINO","TP_ESCOLA","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]]
    return df_selecao

    

 Tentativa da função co loc
def selecao(dff):
    df_selecao = dff.loc[:,["NU_INSCRICAO","NU_ANO","CO_MUNICIPIO_RESIDENCIA","NO_MUNICIPIO_RESIDENCIA","CO_UF_RESIDENCIA","SG_UF_RESIDENCIA","TP_COR_RACA","TP_SEXO","NU_IDADE","TP_ENSINO","TP_ESCOLA","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]]

## Salvar colunas selecionadas

In [ ]:
df_selecao.to_csv('data/selecao.csv',index=False)
df_selecao

## Tirar alunos com nota 0 da selecao

In [ ]:

notas_selecao = df_selecao.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0")
notas_selecao

In [ ]:
# salvar a amostra filtrada em disco
notas_selecao.to_csv('data/enem19-sem_Zzero.csv',index=False)

In [ ]:
#df_selecao = df_selecao[df_selecao["NU_NOTA_MT"] == 0]
#df_selecao

In [ ]:
#df_selecao = df_selecao.drop(notas_zero.index)
#df_selecao

In [ ]:
#Numero pessoas que sairam da analise
len(df_selecao)-len(notas_selecao)

## Salvar dataFrame sem notas 0

In [ ]:

notas_sem_0 = "data/notas_sem_0"
notas_selecao.to_csv(f'{notas_sem_0}.csv',index=False)


In [ ]:
nome_completo = os.path.basename('../inep/enem/2010/DADOS/MICRODADOS_ENEM_2010.csv') #le o nome do arquivo que estamos usando
apenas_ano = nome_completo[-8:-4] #separa o ano dos microdados
apenas_ano

In [ ]:
nome_padronizado = "dados_limpos_enem_" + apenas_ano
nome_padronizado

Não consigo ler o arquivo
teste = pd.read_csv("Enem/data/notas_sem_0.csv")
teste


In [389]:
def limpo(df_selecao):
    df_selecao.to_csv('data/selecao.csv',index=False)
    print(df_selecao)
    

In [390]:
teste = os.path.abspath(df_selecao)
print(f"teste")

teste


In [392]:
def filterdf(df):
    df = df[df["TP_PRESENCA_CN"] == 1]  # presente nas 4 provas
    df = df[df["TP_PRESENCA_CH"] == 1]
    df = df[df["TP_PRESENCA_LC"] == 1]
    df = df[df["TP_PRESENCA_MT"] == 1]
    if "IN_TREINEIRO" in df.columns:
        df = df[(df["IN_TREINEIRO"] == 0) | (df["IN_TREINEIRO"].isna())]  # não é treineiro ou não existe 
    df = df[df["TP_ST_CONCLUSAO"].isin([1, 2])]  # afirma que concluiu ou vai concluir EM em 2019
    # não queremos as notas == 0
    df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0") 
    return df


def load_data(ano,frac):
    'carrega dados com determinado ano e fração'
    print(f"processando {ano} com fraçao = {frac*100:.0f}%",ano, frac)
    def skip(i,fraction=frac):
        if i == 0:
            return False  # precisamos manter o row=0 para ter o header
        else:
            return random.random() > fraction
   
    id
    caminho = f'../inep/enem/{ano}/DADOS/MICRODADOS_ENEM_{ano}.csv'
    #print(caminho)
    df = pd.read_csv(caminho,skiprows = skip, encoding='latin1',sep=";")
    return df
    
def filter_data(df):
    print("filter data...\n")
    return filterdf(df)

def nome_do_arquivo(ano,frac):
    frac = frac*100
    fn = f'data/enem_{frac:.0f}_{ano}.csv'
    return fn

#load_data(2010,0.01)

In [393]:
frac = 0.01
for ano in range(2009,2022):
    df = load_data(ano,frac)
    df = filter_data(df)
    fn = nome_do_arquivo(ano,frac)
    #print(fn+"...\n")
    df.to_csv(fn,index=False)

processando 2009 com fraçao = 1% 2009 0.01
filter data...

processando 2010 com fraçao = 1% 2010 0.01
filter data...

processando 2011 com fraçao = 1% 2011 0.01
filter data...

processando 2012 com fraçao = 1% 2012 0.01
filter data...

processando 2013 com fraçao = 1% 2013 0.01
filter data...

processando 2014 com fraçao = 1% 2014 0.01
filter data...

processando 2015 com fraçao = 1% 2015 0.01
filter data...

processando 2016 com fraçao = 1% 2016 0.01


FileNotFoundError: [Errno 2] No such file or directory: '../inep/enem/2016/DADOS/MICRODADOS_ENEM_2016.csv'

In [ ]:
def load_tri_params(ano):
    'escreve o arequivo com parametros TRI no encoding UTF8'
    fn = f'../inep/enem/{ano}/DADOS/ITENS_PROVA_{ano}.csv'
    print(fn)
    fnconv = f'data/itens_{ano}.csv'
    # ler o arquivo arquivo fn e escrever com enconding utf8
    df = pd.read_csv(fn, sep=';',encoding='latin1')
    df.to_csv(fnconv,index=False)
    return df
    
#load_tri_params(2009)

In [ ]:
for ano in range(2009,2023):
    load_tri_params(ano)